# LLM Testing

Here we test the LLM and check that it works as wanted with the Ollama setup.

Before running this file it is needed to install Ollama and the LLaMA 3.1 model as explained in the `setup_instruction.md`

### First we do a simple call

To check that the LLM is correctly installed and we can query it with python.

In [24]:
# imports
import ollama
import json
from datetime import date

# imports from the api_functions.py file
from api_functions import initialize_session, get_curve, select_data, plot_dataframe

# Reload to get latest version with lag features
import importlib
import analysis_functions
importlib.reload(analysis_functions)
from analysis_functions import extract_features_df, analyze_day


In [25]:
def simple_call(prompt):
    """
    Function to test a simple call to the LLM model using Ollama.

    Args:
        prompt (str): The prompt to send to the LLM.

    Returns:
        str: The response from the LLM.
    """
    response = ollama.chat(
        model="llama3.1:8b",
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        options={
            "temperature": 0.2
        }
    )

    return response["message"]["content"]


In [26]:
# simple test to check if ollama and LLM calls work
prompt = "What color is a strawberry?"
response = simple_call(prompt)
print("Prompt:", prompt)
print("LLM Response:", response)

Prompt: What color is a strawberry?
LLM Response: A strawberry is typically red in color, although some varieties may have a slightly pink or yellow tint to them. The exact shade of red can vary depending on the ripeness and type of strawberry!


### Next we try to implement the Q&A and chat capabilities

In [27]:
# functions I use for the Q&A and chat capabilities

def validate_analysis_request(analysis_request: dict) -> tuple[bool, str]:
    """
    Validate that the analysis request has correct format and allowed values.
    
    Args:
        analysis_request: Dictionary with analysis request fields
    
    Returns:
        tuple: (is_valid: bool, error_message: str)
    """
    # Check required fields exist
    required_fields = ["unclear", "time_frame", "date_from", "date_to", "area", "analysis_type"]
    for field in required_fields:
        if field not in analysis_request:
            return False, f"Missing required field: {field}"
    
    # Validate 'unclear' field
    if analysis_request["unclear"].lower() not in ["yes", "no"]:
        return False, f"Invalid 'unclear' value: must be 'yes' or 'no', got '{analysis_request['unclear']}'"
    
    # Validate 'time_frame' field
    if analysis_request["time_frame"].lower() not in ["day", "longer"]:
        return False, f"Invalid 'time_frame' value: must be 'day' or 'longer', got '{analysis_request['time_frame']}'"
    
    # Validate date format (YYYY-MM-DD)
    import re
    date_pattern = r'^\d{4}-\d{2}-\d{2}$'
    for date_field in ["date_from", "date_to"]:
        if not re.match(date_pattern, analysis_request[date_field]):
            return False, f"Invalid date format for '{date_field}': expected YYYY-MM-DD, got '{analysis_request[date_field]}'"
    
    # Validate 'area' field
    allowed_areas = ["ch", "de", "it", "at", "np", "no", "dk"]
    if analysis_request["area"].lower() not in allowed_areas:
        return False, f"Invalid 'area' value: must be one of {allowed_areas}, got '{analysis_request['area']}'"
    
    # Validate 'analysis_type' field
    allowed_types = ["spot", "intraday"]
    if analysis_request["analysis_type"].lower() not in allowed_types:
        return False, f"Invalid 'analysis_type' value: must be one of {allowed_types}, got '{analysis_request['analysis_type']}'"
    
    # Check date_from <= date_to
    from datetime import datetime
    try:
        date_from_obj = datetime.strptime(analysis_request["date_from"], "%Y-%m-%d")
        date_to_obj = datetime.strptime(analysis_request["date_to"], "%Y-%m-%d")
        if date_from_obj > date_to_obj:
            return False, f"date_from ({analysis_request['date_from']}) must be <= date_to ({analysis_request['date_to']})"
    except ValueError as e:
        return False, f"Invalid date value: {e}"
    
    return True, ""

def get_analysis_request():
    """
    Function that reads the analysis request from the user input.

    Returns:
        dict: A dictionary containing the analysis request details.
    """

    analysis_request_accepted = False

    while not analysis_request_accepted:
        # ask the user to input the request
        print("Write your analysis request below with the day/time frame, area and analysis type (spot/intraday):")
        user_input = input()
        print(user_input)

        analysis_prompt = f"""
            You are an expert energy data analyst. A user has provided you with the following input:

            \"\"\"{user_input}\"\"\"

            Please extract the following information from the user's input:
            1. If the user is interested in a specific day or a longer time period.
            2. The specific day or time period mentioned, if specific day set date_from and date_to as equal to that day. Use the format YYYY-MM-DD. If a longer time period is mentioned, extract the start and end dates.
            3. The country or region (area) the user is interested in, from the following list: (ch, de, it, at, np, no, dk)
            4. The type of analysis the user wants to perform, from the following list: (spot, intraday)

            If any information is missing or unclear, please indicate that in the respective fields.

            Provide the extracted information in a JSON format with the following structure:
            {{
                "unclear": "<yes/no>",
                "time_frame": "<day/longer>",
                "date_from": "<YYYY-MM-DD>",
                "date_to": "<YYYY-MM-DD>",
                "area": "<country_code>",
                "analysis_type": "<spot/intraday>"
            }}
            """



        response = ollama.chat(
            model="llama3.1:8b",
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful assistant, which only outputs JSON in the specified format."
                },
                {
                    "role": "user",
                    "content": analysis_prompt
                }
            ],
            options={
                "temperature": 0.2
            }
        )

        # Parse the JSON response
        try:
            analysis_request = json.loads(response["message"]["content"])
        except json.JSONDecodeError:
            analysis_request = {
                "analysis_type": "",
                "variables": [],
                "time_frame": "",
                "output_format": ""
            }
        
        # check the validity of the analysis request
        is_valid, error_message = validate_analysis_request(analysis_request)
        print("Validation result:", is_valid, error_message)
        if is_valid:
            analysis_request_accepted = True
        else:
            # If not valid, inform the user and loop again
            #print(f"Analysis request not valid: {error_message}. Please try again.")
            print("Could not retrieve all the necessary information from your input.")

    return analysis_request




In [28]:
analysis_request = get_analysis_request()
print("Final analysis request:", analysis_request)

Write your analysis request below with the day/time frame, area and analysis type (spot/intraday):
spot for swiss market for 24 December 2025
Validation result: True 
Final analysis request: {'unclear': 'no', 'time_frame': 'day', 'date_from': '2025-12-24', 'date_to': '2025-12-24', 'area': 'ch', 'analysis_type': 'spot'}


In [29]:
llm_context_info = """
Hourly Rolling Features (from extract_features_df on STL residuals)
    a_mean              : Rolling mean over w hours (baseline level)
    a_std               : Rolling std over w hours (short-term volatility)
    a_mad               : Median Absolute Deviation (robust to outliers)
    a_q25               : 25th percentile over w hours
    a_q75               : 75th percentile over w hours
    a_iqr               : Inter-quartile range (robust spread metric)
    a_z_abs             : Absolute z-score (how many std from mean)
    a_anomaly_rate      : % of hours in window with |z| > z_thr (default 3.0)
    a_rz_abs            : Robust z-score using MAD (resistant to extreme values)
    a_vol_rolling       : Rolling volatility (= std)
    a_vol_ewma          : Exponential moving avg of absolute deviations (recent vol emphasis)
    a_acf1              : Autocorrelation at lag 1 (t vs t-1)
    a_acf2              : Autocorrelation at lag 2 (t vs t-2)
    a_lag24             : Price value from 24 hours ago
    a_lag168            : Price value from 168 hours ago (7 days)
    a_lag24_delta       : Price change over past 24 hours (t - t-24)
    a_lag168_delta      : Price change over past 168 hours (t - t-168)
    a_dow_hour_z        : Z-score normalized by day-of-week + hour-of-day baseline
                              (removes diurnal/weekly seasonality)
    a_mean_shift        : Absolute change in rolling mean (regime change signal)
    
Calendar Context
    weekday                 : Day of week (0=Monday, 1=Tuesday, ..., 6=Sunday)
    hour                    : Hour of day (0=00:00, 1=01:00, ..., 23=23:00)
    
    === Day-Level Statistics ===
    a_day_mean          : Average price for the target day
    a_day_min           : Minimum price during the target day
    a_day_max           : Maximum price during the target day
    a_day_q10           : 10th percentile of prices that day
    a_day_q50           : Median price that day
    a_day_q90           : 90th percentile of prices that day
    
Price Regime Indicators
    a_neg_share         : Share of hours with negative prices (0.0 to 1.0)
    a_spike_share       : Share of hours exceeding 30-day 95th percentile
    
Intra-Day Volatility
    a_ramp_mean         : Average hour-to-hour price change
    a_ramp_std          : Volatility of hour-to-hour changes
    a_ramp_max_abs      : Largest single-hour price jump (absolute value)
    
Lag-Based Momentum & Seasonality 
    a_lag24_mean        : Average price 24 hours ago (yesterday, same time)
    a_lag168_mean       : Average price 168 hours ago (7 days ago, same weekday)
    a_day_vs_lag24      : Today's avg - Yesterday's avg (day momentum)
    a_day_vs_lag168     : Today's avg - Last week's avg (weekly seasonality)
"""

In [31]:
# initialize the session
session = initialize_session()

# define the curve name
curve_name = 'pri it spot €/mwh cet h a'

# get the curve object
curve = get_curve(session, curve_name)
# select data
data_df = select_data(curve, data_from='2025-01-01', data_to='2026-01-10')


# scegliere giorno
features_dday= analyze_day(
    data_df,
    day=analysis_request['date_from'],
    w=48
)

#print("Features for the day:", features_dday)



Authentication succeeded. Session returned.
Curve fetched correctly! Curve ID: 20766, Type: TIME_SERIES
Selecting data from 2025-01-01 to 2026-01-10...


/opt/anaconda3/envs/hackathon-energy/lib/python3.11/site-packages/volue_insight_timeseries/util.py:147: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  return res.asfreq(self._map_freq(self.frequency))


Retrieved 8976 data points.


In [36]:
summary_prompt = f"""
You are an expert energy data analyst.
You get a table with all the following informations about the spot data residuals for a specific day:

{llm_context_info}

Now consider all the following data in this table:

{features_dday}

Give a short summary about the main finding/information and interesting things you notice about this day. Write a short report about it. Do not give general motivation about the possible reasons of the observed patterns. Try to give some numerical values in the insight, to justify your points.
"""

In [37]:
def summary_call(prompt):
    """
    Function to create a summary about the received data with a simple call to the LLM model using Ollama.

    Args:
        prompt (str): The prompt to send to the LLM.

    Returns:
        str: The response from the LLM.
    """
    response = ollama.chat(
        model="llama3.1:8b",
        messages=[
            {
                "role": "system",
                "content": "You are energy data analyst expert, which reads the data from the residual table and returns the most important insights in a short report."
            },
            {
                "role": "user",
                "content": summary_prompt
            }
        ],
        options={
            "temperature": 0.2
        }
    )

    return response["message"]["content"]

summary_response = summary_call(summary_prompt)
print("Summary Response:", summary_response)

Summary Response: **Summary Report: December 24th**

The data analysis reveals a consistent pattern on December 24th, where multiple indicators point to a significant deviation from expected behavior.

1. **Price-It Spot Price**: The price of electricity is consistently lower than expected, with an average difference of -3.6 €/MWh compared to the lagged value (a_day_vs_lag24).
2. **Lagged Values**: The mean values of lagged indicators (a_lag168_mean and a_lag24_mean) are significantly higher than expected, indicating a strong correlation between these variables.
3. **Day vs Lagged Values**: The difference between the day's value and its corresponding lagged value is consistently negative (-10.2 €/MWh on average), suggesting that the current day's value is lower than its historical equivalent.

These findings suggest that December 24th was an unusual day in terms of electricity price behavior, with prices being significantly lower than expected. The exact reasons behind this phenomenon 